In [40]:
import pandas as pd
from itertools import combinations
from itertools import chain
import numpy as np

df = pd.read_csv('data1.csv', header = None)

In [41]:
df

,0,1,2,3,4,5
0,Wine,Chips,Bread,Butter,Milk,Apple
1,Wine,NaN,Bread,Butter,Milk,NaN
2,NaN,NaN,Bread,Butter,Milk,NaN
3,NaN,Chips,NaN,Butter,NaN,Apple
4,Wine,Chips,Bread,Butter,Milk,Apple
5,Wine,Chips,NaN,NaN,Milk,NaN
6,Wine,Chips,Bread,Butter,NaN,Apple
7,Wine,Chips,NaN,NaN,Milk,NaN
8,Wine,NaN,Bread,NaN,NaN,Apple
9,Wine,NaN,Bread,Butter,Milk,NaN


# Apriori

In [5]:
def convert(df):
    rule = {0: 'Wine', 1: 'Chips', 2: 'Bread', 3: 'Butter', 4: 'Milk', 5: 'Apple'}
    for col in df.columns:
        df[col].replace(rule[col], 1, inplace = True)
        df[col].replace(np.nan, 0, inplace = True)
    df.rename(columns = rule, inplace = True)
    return df

In [7]:
df = convert(df)
df

,Wine,Chips,Bread,Butter,Milk,Apple
0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,1.0,1.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,0.0,0.0,1.0,0.0
6,1.0,1.0,1.0,1.0,0.0,1.0
7,1.0,1.0,0.0,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,0.0,1.0
9,1.0,0.0,1.0,1.0,1.0,0.0


In [16]:
def distinct(candidate):
  item_str = ', '.join(candidate)
  return np.unique(item_str.split(', '))

In [17]:
def supp(dist_item, df, k = 1):
  comb_list = combinations(dist_item, k)
  res = []
  for item in comb_list:
    item = list(item)
    occure = (np.sum(df[item], axis = 1) == k)
    frequent = np.sum(occure)
    res.append([', '.join(item),frequent/len(df)])
  return pd.DataFrame(res)

In [18]:
def conf(candidate, k, df):
  res = []
  candidate = candidate.to_numpy()
  for i in range(len(candidate)):
    dist_candidate = candidate[i].split(', ')
    supp_all = supp(dist_candidate, df, k)[1].values
    for j in range(k - 1, 0, -1):

      for left_hand in combinations(dist_candidate, j):
        left_hand = list(left_hand)
        supp_left = supp(left_hand, df, j)[1].values
        right_hand = [i for i in dist_candidate if i not in left_hand]
        left_hand = ', '.join(left_hand)
        right_hand = ', '.join(right_hand)
        res.append([left_hand, right_hand, (supp_all/supp_left)[0]])

  return res

In [19]:
def apriori(df, minsup):
  res = []
  k = 1
  supp_mtx = supp(df.columns, df)
  f = supp_mtx[supp_mtx[1] >= minsup]
  while len(f) != 0:
    k += 1
    dist_item = distinct(f[0])
    supp_mtx = supp(dist_item, df, k)
    res.append(conf(f[0], k - 1, df))
    f = supp_mtx[supp_mtx[1] >= minsup]
  return res

In [35]:
def inspect(result, item_num):
    res = result[item_num - 1]
    return pd.DataFrame(res, columns = ['Left Hand', 'Right Hand', 'Confidence']).sort_values(by = 'Confidence', ascending = False)

In [39]:
res = apriori(df, 0.5)
inspect(res, 3)

,Left Hand,Right Hand,Confidence
0,"Bread, Butter",Milk,0.846154
2,"Butter, Milk",Bread,0.846154
7,"Bread, Wine",Milk,0.846154
1,"Bread, Milk",Butter,0.785714
6,"Bread, Milk",Wine,0.785714
8,"Milk, Wine",Bread,0.785714
4,Butter,"Bread, Milk",0.687500
11,Wine,"Bread, Milk",0.687500
3,Bread,"Butter, Milk",0.647059
9,Bread,"Milk, Wine",0.647059


# Vertical Apriori

In [42]:
def convert(df):
  res = []
  for col in df.columns:
    res.append([[col],[index for index in range(len(df)) if df[col].isnull()[index] == False]])
  return res

In [43]:
df = convert(df)
df

[[[0], [0, 1, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 18, 19, 20]],
 [[1], [0, 3, 4, 5, 6, 7, 10, 12, 15, 16, 17, 18, 20, 21]],
 [[2], [0, 1, 2, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21]],
 [[3], [0, 1, 2, 3, 4, 6, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19]],
 [[4], [0, 1, 2, 4, 5, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]],
 [[5], [0, 3, 4, 6, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20]]]

In [44]:
def conf(data, candidate, k):
  res = []
  for indices in candidate:
    all_supp = len(f_cal(data, indices, k)[0][1])
    for i in range(1, k):
      for left_hand in combinations(indices, i):
        left_supp = len(f_cal(data, left_hand, i)[0][1])
        right_hand = [item for item in indices if item not in left_hand]
        res.append([list(left_hand), right_hand, all_supp/left_supp])
  return res

In [45]:
def distinct(item_list):
  return np.unique(np.array(item_list)).reshape(-1)

In [46]:
def f_cal(data, candidate, k):
  res = []
  for i in combinations(candidate, k):
    id_arr = [data[item][1] for item in i]
    id_arr = np.array(list(chain.from_iterable(id_arr)))
    res.append([list(i), [id for id in data[i[0]][1] if ( (id_arr == id).sum() == k )]] )
  return res

In [47]:
def vert_apriori(data, minsupp):
  res = []
  candidate = np.arange(len(data))
  f = data
  k = 1
  f = [id_list for id_list in f if len(id_list[1]) >= minsupp]
  while (len(f) != 0):
    k += 1
    dist_candidate = distinct(candidate)
    f = f_cal(data, dist_candidate, k)
    f = [id_list for id_list in f if len(id_list[1]) >= minsupp]
    candidate = [item[0] for item in f]
    res.append(conf(data, candidate, k))
  return res, k - 1

In [63]:
def inspect(result, item_num):
  item_list = {0 : 'Wine', 1 : 'Chips', 2 : 'Bread', 3 : 'Butter', 4 : 'Milk', 5 : 'Apple'}
  if (item_num - 2) >= 0:
    res = [i.copy() for i in result[item_num - 2]]
    for asso_rule in res:
      asso_rule[0] = [item_list[indices] for indices in asso_rule[0]]
      asso_rule[0] = ', '.join(asso_rule[0])

      asso_rule[1] = [item_list[indices] for indices in asso_rule[1]]
      asso_rule[1] = ', '.join(asso_rule[1])

    return pd.DataFrame(res, columns = ['Left Hand', 'Right Hand', 'Confidence']).sort_values(by = 'Confidence', ascending = False)

In [64]:
result, max_item = vert_apriori(df, 11)

In [65]:
inspect(result, 3)

,Left Hand,Right Hand,Confidence
3,"Wine, Bread",Milk,0.846154
9,"Bread, Butter",Milk,0.846154
11,"Butter, Milk",Bread,0.846154
4,"Wine, Milk",Bread,0.785714
5,"Bread, Milk",Wine,0.785714
10,"Bread, Milk",Butter,0.785714
0,Wine,"Bread, Milk",0.687500
7,Butter,"Bread, Milk",0.687500
1,Bread,"Wine, Milk",0.647059
6,Bread,"Butter, Milk",0.647059
